# MNIST EXAMPLE WITH CCNS

Copied from https://www.tensorflow.org/tutorials/layers


In [1]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [3]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-u

INFO:tensorflow:loss = 2.293834, step = 1
INFO:tensorflow:probabilities = [[0.1164431  0.10185584 0.09994043 0.09018691 0.10274342 0.08836033
  0.09246649 0.09991439 0.10056967 0.10751943]
 [0.10636708 0.09641965 0.1020744  0.10324452 0.08260914 0.09844325
  0.10676502 0.09452686 0.10867199 0.1008781 ]
 [0.11136676 0.09573934 0.10940053 0.10584095 0.09971002 0.09158053
  0.08769882 0.09700176 0.09330786 0.10835344]
 [0.12937841 0.07215784 0.09627847 0.11951952 0.11124313 0.08882691
  0.09512495 0.08471767 0.09899528 0.10375782]
 [0.12178431 0.09967563 0.08689001 0.09065624 0.10423923 0.08889047
  0.09469178 0.09640247 0.11280812 0.10396182]
 [0.1052901  0.0829621  0.10086337 0.10783698 0.10765781 0.08475407
  0.10317975 0.09870654 0.10097744 0.10777178]
 [0.11617623 0.09764597 0.08870638 0.09797385 0.08975104 0.1068319
  0.09545203 0.09221863 0.11774921 0.09749484]
 [0.11623811 0.09056037 0.09586862 0.09990296 0.10752556 0.08844642
  0.10515845 0.09043224 0.10122968 0.10463751]
 [0.133

INFO:tensorflow:global_step/sec: 7.08238
INFO:tensorflow:probabilities = [[0.13073651 0.09622256 0.09937695 0.10717025 0.09350751 0.09113389
  0.09469001 0.09929944 0.09312441 0.09473842]
 [0.11416133 0.10585135 0.08823209 0.11009856 0.09049721 0.09759562
  0.09578864 0.09903675 0.10597343 0.09276506]
 [0.11787925 0.08000799 0.09262828 0.1135538  0.10695499 0.08689941
  0.10949484 0.09797826 0.0865171  0.10808613]
 [0.11441637 0.10410795 0.09690499 0.09432352 0.10597273 0.0913074
  0.10248063 0.09266302 0.09419162 0.10363182]
 [0.1089044  0.09502347 0.0959097  0.09860328 0.09844458 0.09648969
  0.09701302 0.10747945 0.1074687  0.0946637 ]
 [0.11674965 0.08499805 0.09835609 0.10523709 0.10753509 0.08767211
  0.09389272 0.10657606 0.09923761 0.09974566]
 [0.14018555 0.09275279 0.09011047 0.11728367 0.07799903 0.08232873
  0.10518269 0.09616263 0.08912965 0.1088648 ]
 [0.11864782 0.08732912 0.08831594 0.10255068 0.09674554 0.09163418
  0.1090757  0.09978427 0.10374291 0.10217388]
 [0.1100

INFO:tensorflow:loss = 2.2742712, step = 101 (14.126 sec)
INFO:tensorflow:probabilities = [[0.11932211 0.08738288 0.09828039 0.10011818 0.11951025 0.09078291
  0.09052094 0.08817481 0.10313085 0.10277677]
 [0.10103947 0.09287263 0.10620397 0.09924845 0.11704623 0.08844745
  0.09532849 0.09472322 0.10258796 0.1025021 ]
 [0.13897869 0.09904768 0.09989302 0.10247865 0.0949074  0.08014324
  0.08796795 0.09877796 0.10881473 0.0889907 ]
 [0.12925011 0.0963638  0.08676878 0.0982172  0.09308076 0.0987182
  0.10568084 0.09969259 0.10526208 0.08696564]
 [0.12294903 0.10008654 0.08900855 0.10471848 0.09970172 0.08644263
  0.09624435 0.08453856 0.11091245 0.10539763]
 [0.10511091 0.09651452 0.09389737 0.09547333 0.09209967 0.09855054
  0.09610955 0.11182989 0.11451384 0.09590037]
 [0.13734467 0.0831245  0.10047456 0.09861661 0.10121724 0.08644942
  0.08966941 0.09952983 0.11941519 0.08415858]
 [0.13115473 0.08526654 0.08696678 0.11357656 0.10086194 0.08272942
  0.09686057 0.1004634  0.09844059 0.1

INFO:tensorflow:global_step/sec: 6.45769
INFO:tensorflow:probabilities = [[0.129968   0.08794572 0.09473336 0.11044028 0.09047073 0.08967063
  0.09089006 0.10064957 0.10150579 0.10372587]
 [0.128734   0.08276162 0.08795571 0.1041138  0.08741772 0.1042284
  0.10165745 0.0908157  0.11285065 0.09946498]
 [0.11323287 0.10068063 0.09156497 0.10757694 0.09723759 0.08217378
  0.10555634 0.09922997 0.11157031 0.09117661]
 [0.11273809 0.086753   0.0902927  0.11365674 0.09983334 0.08728628
  0.10573876 0.08295094 0.11363643 0.1071137 ]
 [0.12615068 0.10018926 0.09048043 0.09891587 0.10595241 0.08185596
  0.0945096  0.09661857 0.10701913 0.09830808]
 [0.12588747 0.09079755 0.09001482 0.09795864 0.10140419 0.08742561
  0.11234543 0.09970027 0.09311733 0.10134865]
 [0.13668372 0.06414144 0.08504818 0.10653409 0.10612349 0.09876189
  0.1164951  0.08527099 0.09292307 0.10801789]
 [0.12304492 0.09392757 0.09474467 0.12114545 0.10250107 0.10647126
  0.08844481 0.08989151 0.09303994 0.08678887]
 [0.1248

...

INFO:tensorflow:global_step/sec: 6.18901
INFO:tensorflow:probabilities = [[0.00046345 0.00001247 0.00000514 0.0017935  0.00094531 0.00144661
  0.00000054 0.9836165  0.00044981 0.01126655]
 [0.01043593 0.00164632 0.00440593 0.19187245 0.00263511 0.02648465
  0.00492239 0.00101564 0.73028105 0.02630051]
 [0.00001424 0.00002289 0.00007179 0.00014134 0.8095638  0.00172407
  0.00060142 0.00095454 0.00359616 0.18330967]
 [0.00000581 0.00000013 0.00597892 0.00001893 0.00062177 0.00045714
  0.9924757  0.00000679 0.00040845 0.00002647]
 [0.00077168 0.00000926 0.9923069  0.00047148 0.00001319 0.0001302
  0.00068721 0.00045167 0.0050616  0.00009685]
 [0.00039104 0.00000078 0.00017579 0.00002426 0.0001556  0.00222918
  0.9966223  0.00000016 0.00036048 0.00004032]
 [0.00105811 0.9334133  0.00879985 0.00188943 0.00114401 0.00071454
  0.00071098 0.00446112 0.04742257 0.00038592]
 [0.00152345 0.01132467 0.00705306 0.24129935 0.06328967 0.35408446
  0.00664672 0.00821729 0.15521954 0.15134172]
 [0.0004

INFO:tensorflow:loss = 0.31394166, step = 4001 (16.158 sec)
INFO:tensorflow:probabilities = [[0.0000889  0.00167222 0.00587838 0.01596977 0.00012638 0.00008405
  0.00002221 0.9439467  0.00248453 0.02972681]
 [0.02671831 0.00055681 0.0008216  0.63544756 0.00004625 0.09003137
  0.00004016 0.00325813 0.23948732 0.00359245]
 [0.00089472 0.00050752 0.00010347 0.00048356 0.0001029  0.99132204
  0.00060233 0.00005308 0.00531709 0.00061328]
 [0.00447014 0.8452984  0.01396571 0.02285326 0.0057988  0.0123009
  0.01125496 0.03726485 0.02458062 0.02221231]
 [0.00002507 0.01220156 0.0001226  0.04203277 0.00059122 0.00401094
  0.0000016  0.85103935 0.01275755 0.0772173 ]
 [0.00023866 0.00281195 0.00020519 0.00803436 0.03136969 0.00718217
  0.00023386 0.01762856 0.00964548 0.92265016]
 [0.00040003 0.00025369 0.00128908 0.00015743 0.9830315  0.00007006
  0.00120337 0.00112923 0.00024908 0.01221654]
 [0.00312501 0.01891382 0.03276068 0.00154675 0.0012435  0.00530706
  0.91949075 0.00009665 0.01696132 0

INFO:tensorflow:global_step/sec: 6.25159
INFO:tensorflow:probabilities = [[0.00013785 0.00063747 0.00004211 0.02692349 0.36724097 0.18532732
  0.00149697 0.01162997 0.01626694 0.39029697]
 [0.00050875 0.00190105 0.0045054  0.9547339  0.00006493 0.02961717
  0.00087806 0.0003862  0.00714552 0.00025913]
 [0.00021365 0.4244728  0.04739762 0.08271997 0.01823891 0.08646061
  0.09161349 0.00117426 0.18822184 0.05948692]
 [0.01385738 0.00108093 0.00450183 0.00062242 0.00468862 0.8058204
  0.15153585 0.00001144 0.01745635 0.00042481]
 [0.00011876 0.97904885 0.00012048 0.00057538 0.00017726 0.00076167
  0.00046378 0.00298103 0.01400714 0.0017457 ]
 [0.00005561 0.00000373 0.00009016 0.97812223 0.00000114 0.0196294
  0.00000078 0.00001594 0.00194216 0.00013884]
 [0.00015296 0.00000335 0.00007755 0.00031517 0.28357902 0.00097308
  0.00003399 0.63929415 0.00604913 0.06952161]
 [0.00002401 0.00001491 0.00000387 0.00122272 0.01515444 0.00169055
  0.00003212 0.01336219 0.00059944 0.9678958 ]
 [0.00022

INFO:tensorflow:loss = 0.28825873, step = 4101 (15.998 sec)
INFO:tensorflow:probabilities = [[0.00097273 0.02323509 0.02213776 0.12832558 0.00130784 0.08817197
  0.00128141 0.00979747 0.69851553 0.02625458]
 [0.00035004 0.00012086 0.00002989 0.00014037 0.00000228 0.00126803
  0.0000194  0.00006898 0.996588   0.0014122 ]
 [0.00010058 0.00023451 0.00016777 0.00767231 0.1298229  0.02392151
  0.00005904 0.03163924 0.02957811 0.77680403]
 [0.00020866 0.00915101 0.00454509 0.09565029 0.00863459 0.01551008
  0.00326839 0.0043846  0.8492242  0.00942314]
 [0.00026969 0.00000007 0.00021481 0.9932399  0.         0.00621934
  0.00000019 0.00001593 0.00003978 0.00000024]
 [0.00000151 0.0000003  0.00000682 0.00001051 0.9875234  0.00029147
  0.00989732 0.00003107 0.0005105  0.00172703]
 [0.00027971 0.00027832 0.0001924  0.99109167 0.00000075 0.00752036
  0.00002278 0.00000956 0.00058253 0.00002197]
 [0.03748076 0.00051806 0.77350754 0.08436681 0.00159054 0.02253447
  0.07673448 0.00000677 0.00318082 

KeyboardInterrupt: 